In [1]:
# IMPORT NEEDED PACKAGE
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

file_path = '/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv'
df = pd.read_csv(file_path)
display(df.head())

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
import pandas as pd

# Replace 'path/to/your/file.csv' with the actual path to your file in Google Drive
file_path = '/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv'

try:
    df = pd.read_csv(file_path)
    display(df.head())
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.head

<bound method NDFrame.head of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>

In [6]:
df.shape

(50000, 2)

In [7]:
df["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


# ONE HOT ENCODING

In [8]:
# LABEL ENCODER

df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [9]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# DATA PREPROCESSING

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
train_data, test_data = train_test_split(df, test_size = 0.2, random_state=42)


In [12]:
train_data.shape

(40000, 2)

In [13]:
test_data.shape

(10000, 2)

In [14]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data['review'])

In [15]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [16]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [17]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [18]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [19]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [20]:
Y_test

,sentiment
33553,1
9427,1
199,0
12447,1
39489,0
...,...
28567,0
25079,1
18707,1
15200,0


In [21]:
# MODEL BUILDING

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = "sigmoid"))

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [27]:
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 498s 393ms/step - accuracy: 0.7392 - loss: 0.5130 - val_accuracy: 0.8302 - val_loss: 0.3985
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 490s 392ms/step - accuracy: 0.8597 - loss: 0.3387 - val_accuracy: 0.8283 - val_loss: 0.3948
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 507s 396ms/step - accuracy: 0.8740 - loss: 0.3068 - val_accuracy: 0.8751 - val_loss: 0.3127
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 490s 392ms/step - accuracy: 0.9015 - loss: 0.2460 - val_accuracy: 0.8801 - val_loss: 0.2896
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 499s 399ms/step - accuracy: 0.9063 - loss: 0.2289 - val_accuracy: 0.8926 - val_loss: 0.2687
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 493s 395ms/step - accuracy: 0.9259 - loss: 0.1878 - val_accuracy: 0.8880 - val_loss: 0.2897
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 490s 392ms/step - accuracy: 0.9379 - loss: 0.1617 - val_accuracy: 0.8925 - val_loss: 0.2926
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 489s 391ms/step - ac

In [29]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 115ms/step - accuracy: 0.8808 - loss: 0.3682


In [30]:
print(loss)

0.3645995259284973


In [31]:
print(accuracy)

0.8828999996185303


# Building Predictive System

In [32]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequences = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequences)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [33]:
predictive_system("This movie is fantastic!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step


'positive'

In [34]:
model.save("model.h5")

In [35]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']